In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [2]:
### Step 1. 크롤링할 블로그 url 수집

# 검색어
keyword1 = input("1. 크롤링할 키워드를 입력하세요: ")

driver = webdriver.Chrome('/Users/parkjuhyeon/chromedriver')
driver.get("http://www.naver.com")
time.sleep(2)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

1. 크롤링할 키워드를 입력하세요: 취미


<ipython-input-2-fa4139b793cc>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/parkjuhyeon/chromedriver')
<ipython-input-2-fa4139b793cc>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query") #f12눌러서 확인


In [3]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click()
time.sleep(0.5)

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector(".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

<ipython-input-3-13cf99a2e50b>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("VIEW").click()
<ipython-input-3-13cf99a2e50b>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("블로그").click()
<ipython-input-3-13cf99a2e50b>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("옵션").click()
<ipython-input-3-13cf99a2e50b>:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  item_li = driver.find_elements_by_css_selector(".option .txt")


전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기
3개월


In [4]:
# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 0
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

# 블로그 글 url들 수집
url_list = []
title_list = []

In [5]:
# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

<ipython-input-5-fba5bbc5dff8>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")


url개수:  30
title개수:  30


In [6]:
### Step 2. 블로그 내용 크롤링

# URL이 저장된 리스트 불러오기
for url_load in url_list:
        print(url_load)
        
print("수집한 url 개수: ")
print(len(url_list))

https://blog.naver.com/ihsboy?Redirect=Log&logNo=222576659071
https://blog.naver.com/el512?Redirect=Log&logNo=222556237234
https://blog.naver.com/arang1981?Redirect=Log&logNo=222575765116
https://blog.naver.com/sunken09?Redirect=Log&logNo=222576571283
https://blog.naver.com/hohuk212?Redirect=Log&logNo=222573981788
https://blog.naver.com/tameones?Redirect=Log&logNo=222542507504
https://blog.naver.com/cupcolor?Redirect=Log&logNo=222576173709
https://blog.naver.com/kke5825?Redirect=Log&logNo=222519536560
https://blog.naver.com/wcw2070?Redirect=Log&logNo=222576766594
https://blog.naver.com/onlyeunah?Redirect=Log&logNo=222575396709
https://blog.naver.com/morgunlee?Redirect=Log&logNo=222538100299
https://blog.naver.com/star_296?Redirect=Log&logNo=222552368204
https://blog.naver.com/gluck317?Redirect=Log&logNo=222577106943
https://blog.naver.com/fayefaye79?Redirect=Log&logNo=222574499695
https://blog.naver.com/pukka1004?Redirect=Log&logNo=222548521418
https://blog.naver.com/powerppodea?Redire

In [21]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇 선언
number = len(url_list)    # 수집할 글 갯수


# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_list[i]
    driver = webdriver.Chrome('/Users/parkjuhyeon/chromedriver')
    driver.get(url)
    
    
    # 크롤링
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                        
        tit = driver.find_element_by_css_selector(overlays)
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        date = driver.find_element_by_css_selector(overlays)
        datetime = date.text
        
        # 해쉬태그 크롤링 
           #overlays = "#tagList_222556237234 > a:nth-child(2) > span"
           #tag = driver.find_elements_by_css_selector(overlays)
           #hashtag = tag.text
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
           #target_info['hashtag'] = hashtag
        

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링 성공하면 글 제목을 출력
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue


<ipython-input-21-f144feeea589>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


  0%|          | 0/30 [00:00<?, ?it/s]

<ipython-input-21-f144feeea589>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/parkjuhyeon/chromedriver')
<ipython-input-21-f144feeea589>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tit = driver.find_element_by_css_selector(overlays)
<ipython-input-21-f144feeea589>:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  nick = driver.find_element_by_css_selector(overlays)
<ipython-input-21-f144feeea589>:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  date = driver.find_element_by_css_selector(overlays)
<ipython-input-21-f144feeea589>:37: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  tag = driver.find_elements_by_css_selector(overlays)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.55)
Stacktrace:
0   chromedriver                        0x0000000106af9ff9 __gxx_personality_v0 + 584201
1   chromedriver                        0x0000000106a85403 __gxx_personality_v0 + 106003
2   chromedriver                        0x0000000106641e38 chromedriver + 171576
3   chromedriver                        0x00000001066230fe chromedriver + 45310
4   chromedriver                        0x000000010669831c chromedriver + 525084
5   chromedriver                        0x000000010669c74e chromedriver + 542542
6   chromedriver                        0x00000001066949f8 chromedriver + 510456
7   chromedriver                        0x000000010666a82e chromedriver + 337966
8   chromedriver                        0x000000010666bb65 chromedriver + 342885
9   chromedriver                        0x0000000106ab5f1b __gxx_personality_v0 + 305451
1

In [10]:
print('수집한 글 갯수: ', len(dict))

dict[0]

# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

수집한 글 갯수:  0


KeyError: 0